# PubMedQA inference + Reflexion (Colab-ready)

This notebook is a Colab-friendly conversion of `pubmed_inference_colab.py` and adds an inline reflexion loop: when the model's first 1-token label is incorrect, we ask the model to reflect briefly and then retry the label generation with the reflection appended to the prompt.

This version uses the Unsloth FastLanguageModel (or another Hugging Face-compatible tokenizer/model that supports `generate` and `tokenizer.apply_chat_template`). No OpenAI API is required.

Notes:
- If you upload the whole repo to Colab, you can also import `hotpotqa_runs` helpers; here the notebook is self-contained and avoids LangChain to reduce dependencies.
- Keep `N` small for a smoke test (e.g., 5) before scaling up.
markdown
markdown
# PubMedQA inference + Reflexion (Colab-ready)

This notebook reproduces the PubMed inference flow and demonstrates an inline reflexion strategy (reflect-then-retry) using an Unsloth / Hugging Face model on Google Colab. It avoids OpenAI and LangChain so it is quicker to run in Colab.

Workflow:
- Load model + tokenizer (Unsloth FastLanguageModel or compatible HF tokenizer).
- Run single-token label classification for PubMedQA (yes/no/maybe) by generating exactly 1 token.
- If the first label is incorrect, ask the model to produce a short reflection and retry the label with the reflection included.

Keep `N` small for a smoke test (e.g., 5).
code
python
# Install dependencies (run once in Colab).
# Adjust package list if you already have some packages installed.
!pip install -q unsloth datasets transformers accelerate textstat evaluate rouge_score scikit-learn tqdm
markdown
markdown
## Load model and tokenizer
The notebook uses Unsloth's `FastLanguageModel` which is convenient in Colab. If you prefer a raw Hugging Face model/tokenizer, replace this cell accordingly.
code
python
# Load model and tokenizer (Unsloth)
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)

# Choose model: if the exact model isn't available, replace with another Unsloth or HF model name.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct',
    max_seq_length = 8192,
    load_in_4bit = True,
)

# Apply chat template for consistent chat-style tokenization (same as original script)
tokenizer = get_chat_template(
    tokenizer,
    chat_template = 'llama-3.1',
    mapping = { 'role': 'from', 'content': 'value', 'user': 'human', 'assistant': 'gpt' },
)

# Enable inference mode helper (Unsloth)
FastLanguageModel.for_inference(model)
print('Model and tokenizer loaded')
markdown
markdown
## Helpers: prompts, parsing, and probability extraction
These utilities reproduce the single-token classification behavior in the original script.
code
python
import re, gc, torch
from datasets import load_dataset
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report
import textstat

SYSTEM  = 'Answer ONLY with one word: yes, no, or maybe.'
CLASSES = ['yes','no','maybe']
CLASS_SET = set(CLASSES)
MAX_INP = 768

def build_messages(q, ctx):
    if isinstance(ctx, list): ctx = ' '.join(ctx)
    return [{
        'from': 'human',
        'value': (
            f"{SYSTEM}\n\nQuestion: {q}\n\nAbstract:\n{ctx}\n\n
,
,

,
,
,
1e-12
,
,
,